# Εργασία mongodb
## Παπαδόπουλος Γεώργιος (AM: mpksa20010)
### Σύνδεση στη βάση δεδομένων

In [4]:
import pymongo
import pandas as pd
from pprint import pprint

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["imdb"]

def pretty(res):
    for doc in res:
        pprint(doc)

def prettyFast(res):
    i = 0
    for doc in res:
        pprint(doc)
        i = i + 1
        if i == 50:
            break

### Ανέβασμα δεδομένων

In [11]:
# movies collection
collection = db["movies"]
collection.delete_many({})

csvfile = pd.read_csv('ml-latest/movies.csv' , quotechar='"', encoding='ISO-8859-1').to_dict('records')
for item in csvfile:
    item["genres"] = str(item["genres"]).split("|")
collection.insert_many(csvfile)
print("movies...done")

movies...done


![movies](img/movies.png)

In [ ]:
# tags collection
collection = db["tags"]
collection.delete_many({})

csvfile = pd.read_csv('ml-latest/tags.csv', quotechar='"', encoding='ISO-8859-1').to_dict('records')
collection.insert_many(csvfile)
print("tags...done")

![tags](img/tags.png)

In [ ]:
# genome-tags collection
collection = db["genome-tags"]
collection.delete_many({})

csvfile = pd.read_csv('ml-latest/genome-tags.csv', quotechar='"', encoding='ISO-8859-1').to_dict('records')
collection.insert_many(csvfile)
print("genome-tags...done")

![genome-tags](img/genome-tags.png)

In [ ]:
# genome-scores collection
collection = db["genome-scores"]
collection.delete_many({})

csvfile = pd.read_csv('ml-latest/genome-scores.csv', quotechar='"', encoding='ISO-8859-1').to_dict('records')
collection.insert_many(csvfile)
print("genome-scores...done")

![genome-scores](img/genome-scores.png)

In [4]:
# ratings collection (commented out insert so it is not executed by accident, because this dataset is huge and takes about an hour to complete.
collection = db["ratings"]
# collection.delete_many({})

# csvfile = pd.read_csv('ml-latest/ratings.csv', quotechar='"', encoding='ISO-8859-1').to_dict('records')
# collection.insert_many(csvfile)
print("ratings...done")

collection ready
Dict ready
ratings...done


![ratings](img/ratings.png)

### Ερώτημα (a)
Βρείτε τους χρήστες που έχουν ψηφίσει πάνω από 10 ταινίες

In [5]:
collection = db["ratings"]
result = collection.aggregate([
    {"$group": {"_id": "$movieId", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gte": 10}}},
])
prettyFast(result)

{'_id': 162082, 'count': 723}
{'_id': 6636, 'count': 992}
{'_id': 468, 'count': 8115}
{'_id': 4608, 'count': 149}
{'_id': 56145, 'count': 3749}
{'_id': 4910, 'count': 24}
{'_id': 162246, 'count': 10}
{'_id': 8759, 'count': 21}
{'_id': 95738, 'count': 17}
{'_id': 6217, 'count': 161}
{'_id': 3056, 'count': 131}
{'_id': 60461, 'count': 46}
{'_id': 4499, 'count': 4757}
{'_id': 82722, 'count': 31}
{'_id': 112582, 'count': 287}
{'_id': 3517, 'count': 24}
{'_id': 133873, 'count': 31}
{'_id': 100452, 'count': 13}
{'_id': 133281, 'count': 115}
{'_id': 3291, 'count': 26}
{'_id': 2452, 'count': 104}
{'_id': 58191, 'count': 308}
{'_id': 8934, 'count': 192}
{'_id': 2125, 'count': 7145}
{'_id': 71619, 'count': 398}
{'_id': 56485, 'count': 19}
{'_id': 8532, 'count': 490}
{'_id': 61269, 'count': 26}
{'_id': 74156, 'count': 696}
{'_id': 165503, 'count': 148}
{'_id': 55373, 'count': 11}
{'_id': 8131, 'count': 803}
{'_id': 1877, 'count': 59}
{'_id': 2177, 'count': 672}
{'_id': 168286, 'count': 159}
{'_id

### Ερώτημα (b)
Βρείτε πόσα tags έχει η κάθε ταινία

Το συγκεκριμένο query πήρε πάνω από μισή ώρα να εκτελεστεί. Δοκιμάστηκε, όμως, μέσα από το MongoDBCompass όπου επιστρέφονται μόνο 20 αποτελέσματα και έτσι έτρεξε πολύ γρηγορότερα.

In [ ]:
collection = db["tags"]
result = collection.aggregate([
    {"$group": {"_id": "$movieId", "count": {"$sum": 1}}},
    {"$lookup": {
        "from": "movies",
        "localField": "_id",
        "foreignField": "movieId",
        "as": "movie"
    }},
    {"$replaceRoot":{
        "newRoot": {
            "$mergeObjects": [
                {"$arrayElemAt": ["$movie", 0]},
                "$$ROOT"
            ]
        }
    }},
    {"$project": {
        "count": 1,
        "title": 1,
        "_id": 0
    }}
])
prettyFast(result)

### Ερώτημα (c)
Βρείτε το µέσο rating ανά ταινία

In [5]:
collection = db["ratings"]
result = collection.aggregate([
    {"$group": {"_id": "$movieId", "averageRating": {"$avg": "$rating"}}},
        {"$lookup": {
        "from": "movies",
        "localField": "_id",
        "foreignField": "movieId",
        "as": "movie"
    }},
    {"$replaceRoot":{
        "newRoot": {
            "$mergeObjects": [
                {"$arrayElemAt": ["$movie", 0]},
                "$$ROOT"
            ]
        }
    }},
    {"$project": {
        "averageRating": 1,
        "title": 1,
        "_id": 0
    }}
])
prettyFast(result)

[{'_id': 137707, 'averageRating': 1.6666666666666667}, {'_id': 8265, 'averageRating': 2.264367816091954}, {'_id': 176439, 'averageRating': 2.0}, {'_id': 157162, 'averageRating': 4.0}, {'_id': 130734, 'averageRating': 2.0}, {'_id': 190295, 'averageRating': 4.5}, {'_id': 174873, 'averageRating': 2.3333333333333335}, {'_id': 6016, 'averageRating': 4.184896558122275}, {'_id': 105073, 'averageRating': 2.7083333333333335}, {'_id': 1620, 'averageRating': 3.3766221897276547}, {'_id': 164801, 'averageRating': 3.111111111111111}, {'_id': 134103, 'averageRating': 2.0}, {'_id': 5323, 'averageRating': 2.1045673076923075}, {'_id': 2389, 'averageRating': 2.8231052036199094}, {'_id': 3427, 'averageRating': 3.2676240208877285}, {'_id': 161177, 'averageRating': 2.7857142857142856}, {'_id': 130972, 'averageRating': 1.5}, {'_id': 86620, 'averageRating': 3.0}, {'_id': 173889, 'averageRating': 3.5}, {'_id': 138921, 'averageRating': 3.0}, {'_id': 179601, 'averageRating': 4.5}, {'_id': 103366, 'averageRating'

### Ερώτημα (d)
Βρείτε κατά µέσο όρο πόσα genres ανατίθενται σε κάποια ταινία

In [6]:
collection = db["movies"]
result = collection.aggregate([
    {"$addFields": {
       "count": {
           "$size": "$genres"
       }
    }},
    {"$group": {
        "_id": None,
        "averageGenreCount": {"$avg": "$count"}
    }},
    {"$project":{
        "_id": 0
    }}
])
prettyFast(result)

{'averageGenreCount': 1.8263451409687081}


### Εισαγωγή indexes
#### Ερώτημα (a)
Σε αυτή την περίπτωση δεν μπορούμε να χρησιμοποιήσουμε τη δύναμη των indexes καθώς σύμφωνα με τα [docs](https://docs.mongodb.com/master/core/aggregation-pipeline/#pipeline-operators-and-indexes)
της mongodb το `$group` stage μπορεί να χρησιμοποιήσει έναν index σε πολύ συγκεκριμένες περιπτώσεις που δεν ικανοποιούνται στην περίπτωσή μας.
Ένω το `$match` stage μπορεί να χρησιμοιπήσει έναν index δυστυχώς πρόκειται για πεδίου που δημιουργείται από το `$group` και έτσι δεν υπάρχει δυνατότητα να δημιουργεί κάποιος index σε αυτό.


#### Ερώτημα (b)
Η δημιουργία index σε αυτό το ερώτημα είναι σωτήρια. Όπως έχει σημειωθεί στο αρχικό ερώτημα (χωρίς index) η διάρκεια εκτέλεσής του ήταν υπερβολικά μεγάλη.
Πλέον εκτελείται σε μερικά δευτερόλεπτα.

In [2]:
db["movies"].create_index([("movieId", 1)])
collection = db["tags"]
collection.create_index([("movieId", 1)])

'movieId_1'

In [7]:
result = collection.aggregate([
    {"$group": {"_id": "$movieId", "count": {"$sum": 1}}},
    {"$lookup": {
        "from": "movies",
        "localField": "_id",
        "foreignField": "movieId",
        "as": "movie"
    }},
    {"$replaceRoot":{
        "newRoot": {
            "$mergeObjects": [
                {"$arrayElemAt": ["$movie", 0]},
                "$$ROOT"
            ]
        }
    }},
    {"$project": {
        "count": 1,
        "title": 1,
        "_id": 0
    }}
])
prettyFast(result)

{'count': 1, 'title': 'Seasons (2016)'}
{'count': 1, 'title': 'Care Bears Movie, The (1985)'}
{'count': 1, 'title': 'Gangsters, Guns and Zombies (2012)'}
{'count': 1, 'title': "Isabella Rossellini's Green Porno Live (2015)"}
{'count': 1, 'title': 'Kid Cannabis (2014)'}
{'count': 1, 'title': 'Passing Strange (2009)'}
{'count': 1, 'title': 'A Perfect Day (2015)'}
{'count': 1, 'title': 'Crimson Petal and the White, The (2011)'}
{'count': 1, 'title': 'Tusk (2014)'}
{'count': 1, 'title': 'The Pharmacist (2003)'}
{'count': 1, 'title': 'Hellbound (1994)'}
{'count': 1,
 'title': 'Three Kingdoms: Resurrection of the Dragon (Saam gwok dzi gin lung '
          'se gap) (2008)'}
{'count': 1, 'title': 'Tie That Binds, The (1995)'}
{'count': 1, 'title': 'Viking, The (1928)'}
{'count': 1, 'title': 'Epitaph (2007)'}
{'count': 1, 'title': 'These Old Broads (2001)'}
{'count': 1, 'title': 'With a Song in My Heart (1952)'}
{'count': 1, 'title': 'Mr. Nobody (2009)'}
{'count': 1, 'title': 'Magnificent Doll 

#### Ερώτημα (c)
Αντίστοιχα με το προηγούμενο ερώτημα και έδω η δημιουργία index ήταν σωτήρια

In [3]:
collection = db["ratings"]
collection.create_index([("movieId", 1)])

'movieId_1'

In [8]:
result = collection.aggregate([
    {"$group": {"_id": "$movieId", "averageRating": {"$avg": "$rating"}}},
    {"$lookup": {
        "from": "movies",
        "localField": "_id",
        "foreignField": "movieId",
        "as": "movie"
    }},
    {"$replaceRoot":{
        "newRoot": {
            "$mergeObjects": [
                {"$arrayElemAt": ["$movie", 0]},
                "$$ROOT"
            ]
        }
    }},
    {"$project": {
        "averageRating": 1,
        "title": 1,
        "_id": 0
    }}
])
prettyFast(result)

{'averageRating': None, 'title': 'The Puzzle of the Red Orchid (1962)'}
{'averageRating': None, 'title': 'Set It Off (1996)'}
{'averageRating': None, 'title': 'Beauties of the Night (1952)'}
{'averageRating': None,
 'title': 'Distant Sky - Nick Cave & The Bad Seeds Live in Copenhagen (2018)'}
{'averageRating': None, 'title': 'Black Horizon (2002)'}
{'averageRating': None, 'title': '13 Hours (2016)'}
{'averageRating': None, 'title': 'Condo Painting (2000)'}
{'averageRating': None, 'title': 'Tears of Steel (2012)'}
{'averageRating': None,
 'title': 'Ugly, Dirty and Bad (Brutti sporchi e cattivi) (1976)'}
{'averageRating': None, 'title': 'Toll of the Sea, The (1922)'}
{'averageRating': None, 'title': 'December Boys (2007)'}
{'averageRating': None, 'title': 'Left to Die (2012)'}
{'averageRating': None, 'title': 'Stealing Paradise (2011)'}
{'averageRating': None, 'title': 'Shortbus (2006)'}
{'averageRating': None, 'title': 'Earth (1998)'}
{'averageRating': None, 'title': 'Urban Legends: Blo

#### Ερώτημα (d)
Και σε αυτό το ερώτημα λόγω τον εντολών που χρησιμοποιούνται δε γίνεται να χρησιμοποιηθούν indexes.